In [1]:
### CELL 1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install Keras-Applications

     |████████████████████████████████| 51kB 3.0MB/s 


In [4]:
### CELL 2
!git clone https://github.com/maniver7/yoga-82.git

fatal: destination path 'yoga-82' already exists and is not an empty directory.


In [ ]:
#### Not needed. Training and validaton data is directly sent from the drive
#import zipfile
#zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Images_cleaned2.zip", 'r')
#zip_ref.extractall("/content/alldata")
#zip_ref.close()

In [ ]:
## DON'T RUN##################
#from keras.optimizers import SGD,Adam
##import tensorflow as tf
#from tensorflow import keras
#from tf.keras import utils
#import keras
#from tf.keras.utils import np_utils
#from tensorflow.keras.utils import get_source_inputs
#from keras.utils import np_utils
#from keras import utils as np_utils
#import numpy as np
#import random
#import cv2
#import os
#import random
#import matplotlib
#matplotlib.use('AGG')
#import matplotlib.pyplot as plt
#from keras.callbacks import ModelCheckpoint,CSVLogger
#import tensorflow as tf
#import keras.backend as K
#from PIL import Image
#from PIL import ImageFile
#ImageFile.LOAD_TRUNCATED_IMAGES = True
#from schedules import onetenth_4_8_12
#from sklearn.metrics import classification_report, confusion_matrix
#import warnings
#warnings.filterwarnings("ignore")

In [5]:
##### CELL 3
num_classes = 82
batch_size = 32
epochs = 50

In [ ]:
!pwd

/content


In [6]:
######### CELL 4
## make changes to keras_densenet_modified.py before running this file. Comment out keras.utils. Instead use from tensorflow.utils.keras
## import get_source_inputs
#import models
import os
# os.getcwd()
os.chdir('/content/yoga-82')
import keras_densenet_modified
import models
#from models import model_one_class, dense201_hirar, dense201_hirar_6same20, dense201_hirar_new
from models import dense201_hirar_new



In [7]:
#### CELL 5
import keras_densenet_modified
model = dense201_hirar_new()

In [8]:
## CELL 6
from keras.optimizers import SGD,Adam
learning_rate = 0.003 # orig= 0.003
lr = 0.003
sgd = SGD(learning_rate=lr, momentum=0.9, nesterov=False)
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)


In [9]:
#### CELL 7
#model.compile(loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'], loss_weights=[1,1,1], optimizer= sgd, metrics=['accuracy'])#, 'top_k_categorical_accuracy'])
model.compile(loss='categorical_crossentropy', loss_weights= 1, optimizer= sgd, metrics=['accuracy'])
#model.compile(loss = 'categorical_crossentropy', loss_weights=[1,1,1], optimizer= sgd, metrics=['accuracy'])

In [10]:
### CELL 8
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [11]:
### cell 9
from keras.models import Model
model2= Model(inputs=model.input, outputs=model.layers[-1].output)

In [12]:
### CELL 10
### From 2nd run....loading best weights
model2.load_weights("/content/drive/MyDrive/weights-improvement-19-0.83.hdf5")
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [13]:
## CELL 11
model2.compile(loss='categorical_crossentropy', loss_weights= 1, optimizer= sgd, metrics=['accuracy'])
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [14]:
### CELL 12... Path to save the best weights and giving the name to the weight file
filepath="/content/drive/MyDrive/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"

In [15]:
#### CELL 13
from keras.callbacks import ModelCheckpoint,CSVLogger
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [16]:
callbacks_list = [checkpoint]

In [17]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/training',
target_size = (224,224),
batch_size = batch_size,
class_mode = 'categorical')

validate_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/validation',
target_size = (224,224),
batch_size = batch_size,
class_mode = 'categorical')

Found 24971 images belonging to 82 classes.
Found 4881 images belonging to 82 classes.


In [19]:
batch_size = 128

In [20]:
model2.fit(train_generator,
steps_per_epoch = (24971//batch_size),
epochs = 20,
validation_data = validate_generator,
callbacks=callbacks_list,
validation_steps = (4881//batch_size))

Epoch 1/20
 74/195 [==========>...................] - ETA: 13:38 - loss: 0.2456 - accuracy: 0.9389

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


195/195 [==============================] - ETA: 0s - loss: 0.2550 - accuracy: 0.9351

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 55 bytes but only got 54. Skipping tag 33432
  " Skipping tag %s" % (size, len(data), tag)


195/195 [==============================] - 1687s 8s/step - loss: 0.2551 - accuracy: 0.9350 - val_loss: 0.6424 - val_accuracy: 0.8199

Epoch 00001: val_accuracy improved from -inf to 0.81990, saving model to /content/drive/MyDrive/weights-improvement-01-0.82.hdf5
Epoch 2/20
195/195 [==============================] - 1194s 6s/step - loss: 0.2539 - accuracy: 0.9298 - val_loss: 0.6159 - val_accuracy: 0.8076

Epoch 00002: val_accuracy did not improve from 0.81990
Epoch 3/20
195/195 [==============================] - 905s 5s/step - loss: 0.2338 - accuracy: 0.9398 - val_loss: 0.6037 - val_accuracy: 0.8240

Epoch 00003: val_accuracy improved from 0.81990 to 0.82401, saving model to /content/drive/MyDrive/weights-improvement-03-0.82.hdf5
Epoch 4/20
195/195 [==============================] - 698s 4s/step - loss: 0.2328 - accuracy: 0.9357 - val_loss: 0.8250 - val_accuracy: 0.7730

Epoch 00004: val_accuracy did not improve from 0.82401
Epoch 5/20
195/195 [==============================] - 568s 3s/

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


195/195 [==============================] - 436s 2s/step - loss: 0.2172 - accuracy: 0.9371 - val_loss: 1.7279 - val_accuracy: 0.5748

Epoch 00006: val_accuracy did not improve from 0.82401
Epoch 7/20
195/195 [==============================] - 365s 2s/step - loss: 0.2222 - accuracy: 0.9402 - val_loss: 0.5545 - val_accuracy: 0.8396

Epoch 00007: val_accuracy improved from 0.82401 to 0.83964, saving model to /content/drive/MyDrive/weights-improvement-07-0.84.hdf5
Epoch 8/20
195/195 [==============================] - 297s 2s/step - loss: 0.2026 - accuracy: 0.9506 - val_loss: 0.6111 - val_accuracy: 0.8150

Epoch 00008: val_accuracy did not improve from 0.83964
Epoch 9/20
195/195 [==============================] - 259s 1s/step - loss: 0.1845 - accuracy: 0.9549 - val_loss: 0.4264 - val_accuracy: 0.8799

Epoch 00009: val_accuracy improved from 0.83964 to 0.87993, saving model to /content/drive/MyDrive/weights-improvement-09-0.88.hdf5
Epoch 10/20
195/195 [==============================] - 212s 1

In [ ]:
#/content/drive/MyDrive/weights-improvement-20-0.40.hdf5
model2.load_weights("/content/drive/MyDrive/weights-improvement-20-0.67.hdf5")

In [ ]:
model2.compile(loss='categorical_crossentropy', loss_weights= 1, optimizer= sgd, metrics=['accuracy'])
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

##########################################END###########################

##################### END ####################################

In [ ]:
#model.fit(X, Y, validation_split=0.330.2, epochs=200, batch_size=32, callbacks=callbacks_list, verbose=0)
 model.fit_generator(generator_train_batch(train_file, batch_size, num_classes,img_path),
                          steps_per_epoch=train_samples // batch_size,
                          epochs=epochs,
                          callbacks=[checkpointer, csv_logger],
                          validation_data=generator_val_batch(test_file, batch_size,num_classes,img_path),
                          validation_steps=test_samples // batch_size,
                          verbose=1)

In [ ]:
import os
import random

import os
if __name__ == "__main__":
    for (root,dirs,files) in os.walk('/content/alldata/Images', topdown=True):
        print (root)
        print (dirs)
        print (files)
        print ('--------------------------------')



In [ ]:
import os
import random

files_list = []
list1 = ['/content/alldata/Images/Akarna_Dhanurasana/43.jpg']
for root, dirs, files in os.walk('/content/alldata/Images/Akarna_Dhanurasana'):
    for file in files:
        if file.endswith(".jpg"):
            files_list.append(os.path.join(root, file))
    
# print 20 random images
#print(random.sample(files_list, 20))
print(len(files_list))
print(list1)
rand_samp = random.sample(files_list, len(files_list)//5)
rand_samp = list(rand_samp)
print(rand_samp)
list1.extend(list(rand_samp))

#print(len(random.sample(files_list, len(files_list)//5)))



104
['/content/alldata/Images/Akarna_Dhanurasana/43.jpg']
['/content/alldata/Images/Akarna_Dhanurasana/159.jpg', '/content/alldata/Images/Akarna_Dhanurasana/146.jpg', '/content/alldata/Images/Akarna_Dhanurasana/17.jpg', '/content/alldata/Images/Akarna_Dhanurasana/173.jpg', '/content/alldata/Images/Akarna_Dhanurasana/3.jpg', '/content/alldata/Images/Akarna_Dhanurasana/55.jpg', '/content/alldata/Images/Akarna_Dhanurasana/160.jpg', '/content/alldata/Images/Akarna_Dhanurasana/24.jpg', '/content/alldata/Images/Akarna_Dhanurasana/2.jpg', '/content/alldata/Images/Akarna_Dhanurasana/112.jpg', '/content/alldata/Images/Akarna_Dhanurasana/56.jpg', '/content/alldata/Images/Akarna_Dhanurasana/95.jpg', '/content/alldata/Images/Akarna_Dhanurasana/101.jpg', '/content/alldata/Images/Akarna_Dhanurasana/26.jpg', '/content/alldata/Images/Akarna_Dhanurasana/155.jpg', '/content/alldata/Images/Akarna_Dhanurasana/5.jpg', '/content/alldata/Images/Akarna_Dhanurasana/79.jpg', '/content/alldata/Images/Akarna_Dhan

In [ ]:
import random
import shutil
list1 = []
for name in os.listdir("/content/alldata/Images/"):
  files_list = []
  print(name)
  for root, dirs, files in os.walk("/content/alldata/Images/"+ name):
    for file in files:
        if file.endswith(".jpg"):
            files_list.append(os.path.join(root, file))
    print(len(files_list))
    print(list1)
    rand_samp = random.sample(files_list, len(files_list)//5)
    rand_samp = list(rand_samp)
    source = "/content/drive/MyDrive/training/"+ name
    dest = "/content/drive/MyDrive/validation" + name
    for i in range(len(rand_samp)-1):
      head, tail = os.path.split(rand_samp[i])
      f_name = tail
      #shutil.move("/content/drive/MyDrive/training/"+ name + "/" + f_name, "/content/drive/MyDrive/validation/"+ name + "/" + f_name)
      print(f_name)

    #print(rand_samp)
    #list1.extend(list(rand_samp))
    

In [ ]:
import os
directory = "training"
directory1 = "validation"
parent_dir = "/content/drive/MyDrive"
path = os.path.join(parent_dir, directory)
path1 = os.path.join(parent_dir, directory1)
os.mkdir(path)
os.mkdir(path1)

FileExistsError: ignored

In [ ]:
for name in os.listdir("/content/alldata/Images/"):
  os.mkdir("/content/drive/MyDrive/training/"+ name)

In [ ]:
for name in os.listdir("/content/alldata/Images/"):
  os.mkdir("/content/drive/MyDrive/validation/"+ name)

In [ ]:
import shutil os
src = '/content/alldata/Images'
dest = '/content/drive/MyDrive/training'
#shutil.copytree(src, dest, ignore = ignoreFunc('a'))
shutil.copytree(src, dest)

FileExistsError: ignored

In [ ]:
import os, shutil
s = '/content/alldata/Images'
dest = '/content/drive/MyDrive/training'
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

In [ ]:
copytree(s, d, symlinks=False, ignore=None)

In [ ]:
import os, shutil
folder = '/content/drive/MyDrive/training'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
import random
import shutil
list1 = []
for name in os.listdir("/content/drive/MyDrive/training/"):
  files_list = []
  for root, dirs, files in os.walk("/content/drive/MyDrive/training/"+ name):
    for file in files:
        if file.endswith(".jpg"):
            files_list.append(os.path.join(root, file))
    print(len(files_list))
    print(list1)
    rand_samp = random.sample(files_list, len(files_list)//5)
    rand_samp = list(rand_samp)
    print(rand_samp)

430
[]
['/content/drive/MyDrive/training/Child_Pose_or_Balasana_/186.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/362.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/574.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/299.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/398.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/492.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/361.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/481.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/377.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/157.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/162.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/161.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/553.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasana_/104.jpg', '/content/drive/MyDrive/training/Child_Pose_or_Balasan

In [ ]:
head, tail = os.path.split(rand_samp[0])
print(tail)

466.jpg


In [ ]:
import keras
keras.__version__

AttributeError: ignored

In [ ]:
!pip install q keras==2.5.0

ERROR: Could not find a version that satisfies the requirement keras==2.5.0 (from versions: 0.2.0, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.0.8, 1.1.0, 1.1.1, 1.1.2, 1.2.0, 1.2.1, 1.2.2, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.0.5, 2.0.6, 2.0.7, 2.0.8, 2.0.9, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.1.5, 2.1.6, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.2.4, 2.2.5, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.5.0rc0)
ERROR: No matching distribution found for keras==2.5.0


In [ ]:
from PIL import Image
import os
for name in os.listdir("/content/drive/MyDrive/training/"):
  print(name)
  for f_name in os.listdir("/content/drive/MyDrive/training/"+ name):
    print(f_name)
    im = Image.open("/content/drive/MyDrive/training/"+ name + "/" + f_name)


Streaming output truncated to the last 5000 lines.
1_52.jpg
1_248.jpg
1_221.jpg
1_154.jpg
1_347.jpg
1_231.jpg
1_110.jpg
1_49.jpg
1_352.jpg
1_473.jpg
1_151.jpg
1_181.jpg
1_296.jpg
1_531.jpg
1_344.jpg
1_330.jpg
1_387.jpg
1_239.jpg
1_302.jpg
2_147.jpg
1_9.jpg
1_71.jpg
1_85.jpg
1_61.jpg
2_374.jpg
2_430.jpg
1_62.jpg
1_93.jpg
2_137.jpg
1_96.jpg
1_70.jpg
1_69.jpg
2_314.jpg
1_86.jpg
2_522.jpg
2_269.jpg
1_8.jpg
1_647.jpg
2_92.jpg
1_67.jpg
1_74.jpg
1_72.jpg
2_495.jpg
1_89.jpg
2_68.jpg
1_87.jpg
1_88.jpg
2_359.jpg
2_412.jpg
2_394.jpg
1_65.jpg
1_83.jpg
2_416.jpg
1_6.jpg
2_548.jpg
2_210.jpg
1_59.jpg
1_64.jpg
1_75.jpg
1_78.jpg
1_73.jpg
1_608.jpg
2_34.jpg
1_92.jpg
1_94.jpg
2_97.jpg
1_602.jpg
2_340.jpg
2_77.jpg
2_334.jpg
2_457.jpg
1_98.jpg
2_30.jpg
1_97.jpg
Scale_Pose_or_Tolasana_
1_131.jpg
1_112.jpg
1_68.jpg
1_149.jpg
1_138.jpg
1_22.jpg
1_71.jpg
1_81.jpg
1_147.jpg
1_132.jpg
1_48.jpg
1_45.jpg
1_49.jpg
1_72.jpg
1_146.jpg
1_167.jpg
1_119.jpg
1_92.jpg
1_46.jpg
1_274.jpg
1_231.jpg
1_141.jpg
1_108.jpg
1_126

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 55 bytes but only got 54. Skipping tag 33432
  " Skipping tag %s" % (size, len(data), tag)


2_437.jpg
1_403.jpg
2_203.jpg
1_33.jpg
1_512.jpg
2_114.jpg
3_56.jpg
3_204.jpg
2_131.jpg
2_20.jpg
2_4.jpg
2_466.jpg
2_311.jpg
1_374.jpg
2_208.jpg
2_553.jpg
1_401.jpg
3_187.jpg
2_160.jpg
2_465.jpg
2_62.jpg
2_109.jpg
2_192.jpg
2_310.jpg
2_468.jpg
2_514.jpg
2_105.jpg
1_467.jpg
2_46.jpg
2_129.jpg
2_66.jpg
3_214.jpg
1_44.jpg
1_462.jpg
2_218.jpg
2_522.jpg
2_16.jpg
1_347.jpg
2_224.jpg
3_315.jpg
2_336.jpg
2_517.jpg
2_86.jpg
1_83.jpg
2_470.jpg
2_9.jpg
1_552.jpg
1_586.jpg
2_609.jpg
1_69.jpg
1_361.jpg
2_60.jpg
2_321.jpg
3_245.jpg
2_28.jpg
2_135.jpg
2_562.jpg
2_63.jpg
2_332.jpg
3_257.jpg
3_81.jpg
1_49.jpg
2_558.jpg
2_307.jpg
2_217.jpg
2_502.jpg
2_471.jpg
2_596.jpg
3_158.jpg
1_75.jpg
1_563.jpg
2_282.jpg
1_476.jpg
3_141.jpg
2_290.jpg
1_354.jpg
2_55.jpg
3_203.jpg
1_70.jpg
2_115.jpg
2_59.jpg
1_490.jpg
2_102.jpg
1_73.jpg
2_518.jpg
1_627.jpg
1_402.jpg
2_351.jpg
1_560.jpg
2_153.jpg
1_519.jpg
2_246.jpg
2_176.jpg
2_580.jpg
2_213.jpg
1_362.jpg
3_207.jpg
3_58.jpg
2_496.jpg
1_522.jpg
2_76.jpg
2_397.jpg
2_417.j

IsADirectoryError: ignored

In [ ]:
from PIL import Image

im = Image.open(r"C:\Users\System-Pc\Desktop\ybear.jpg") 

In [ ]:
##############EXAMPLE OF pathlib
from PIL import Image
import io
import pathlib

with Image.open('test.jpg') as im:
    ...

with Image.open(pathlib.Path('test.jpg')) as im2:
    ...

with open('test.jpg', 'rb') as f:
    im3 = Image.open(f)
    ...

with open('test.jpg', 'rb') as f:
    im4 = Image.open(io.BytesIO(f.read()))
    ...